In [7]:
import cv2
import torch
import numpy as np
from google.colab.patches import cv2_imshow
from ultralytics import YOLO # make sure to import the YOLO class

# Load the YOLOv8 model
model = YOLO('yolov8s.pt')

# Set the confidence threshold and NMS threshold
conf_thres = 0.5
nms_thres = 0.4

# Set the class labels for the vehicles
class_labels = ['car', 'bus', 'motorcycle']

# Define the polygon zone boundaries
zone_boundaries = np.array([[472, 2119], [3384, 2103], [2604, 735], [1436, 711]])

# Create a video capture object
cap = cv2.VideoCapture('Task2.mp4')  # Replace with your video file or camera index

# Initialize the vehicle counts
vehicle_counts = {'car': 0, 'bus': 0, 'motorcycle': 0}

while True:
    # Read a frame from the video stream
    ret, frame = cap.read()

    if not ret:
        break # Exit the loop if there are no more frames in the video

    # Convert the frame to a tensor and resize
    img = cv2.resize(frame, (640, 640)) # Resize the frame to 640x640
    tensor = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).float() / 255.0 # Convert to tensor, normalize and add batch dimension

    # Perform object detection using YOLOv8
    results = model(tensor)

    # Extract the detections
    detections = results[0].boxes.data.cpu().numpy() # Access detections correctly

    # Initialize the frame vehicle counts
    frame_vehicle_counts = {'car': 0, 'bus': 0, 'motorcycle': 0}

    # Iterate through the detections
    for detection in detections: # Iterate directly through detections
        conf, cls_pred, x, y, x2, y2 = detection[:6] # Extract the relevant information


        # Get the class label
        # Check if cls_pred is within the valid range of class_labels
        if 0 <= int(cls_pred) < len(class_labels):
            class_label = class_labels[int(cls_pred)]
        else:
            # Handle the case where cls_pred is out of range
            print(f"Warning: Class index {int(cls_pred)} out of range. Skipping detection.")
            continue

        # Check if the detection is within the polygon zone
        if cv2.pointPolygonTest(zone_boundaries, (int((x+x2)/2), int((y+y2)/2)), False) >= 0:
            # Increment the frame vehicle count
            frame_vehicle_counts[class_label] += 1

            # Increment the total vehicle count
            vehicle_counts[class_label] += 1

            # Draw a bounding box around the detected vehicle
            cv2.rectangle(frame, (int(x), int(y)), (int(x2), int(y2)), (0, 255, 0), 2)

            # Display the class label and confidence
            cv2.putText(frame, f"{class_label} {conf:.2f}", (int(x), int(y-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Draw the polygon zone
    cv2.drawContours(frame, [zone_boundaries], -1, (0, 255, 0), 2)

    # Display the frame vehicle counts
    cv2.putText(frame, f"Frame Vehicle Counts: Car={frame_vehicle_counts['car']}, Bus={frame_vehicle_counts['bus']}, Motorcycle={frame_vehicle_counts['motorcycle']}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Display the total vehicle counts
    cv2.putText(frame, f"Total Vehicle Counts: Car={vehicle_counts['car']}, Bus={vehicle_counts['bus']}, Motorcycle={vehicle_counts['motorcycle']}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow('YOLOv8 Inference', frame)

    # Exit on key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()
cv2.destroyAllWindows()